In [40]:
from pandas import DataFrame, read_csv
import pandas as pd
from six.moves.urllib.request import urlretrieve


url = 'https://s3-ap-northeast-1.amazonaws.com/furbo-mixpenal/kaggle_data/'
trainFile, _ = urlretrieve(url + 'train.csv')
testFile, _ = urlretrieve(url + 'test.csv')

data_train = pd.read_csv(trainFile)
data_test = pd.read_csv(testFile)
data_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [41]:
from sklearn.ensemble import RandomForestRegressor

def set_missing_ages(df):
    age_df = df[['Age','Fare', 'Parch', 'SibSp', 'Pclass']]
    known_age = age_df[age_df.Age.notnull()].as_matrix()
    unknown_age = age_df[age_df.Age.isnull()].as_matrix()
    y = known_age[:, 0]
    X = known_age[:, 1:]
    rfr = RandomForestRegressor(random_state=0, n_estimators=2000, n_jobs=-1)
    rfr.fit(X, y)
    predictedAges = rfr.predict(unknown_age[:, 1::])
    df.loc[ (df.Age.isnull()), 'Age' ] = predictedAges 
    return df, rfr

def set_Cabin_type(df):
    df.loc[ (df.Cabin.notnull()), 'Cabin' ] = "Yes"
    df.loc[ (df.Cabin.isnull()), 'Cabin' ] = "No"
    return df

data_train, rfr = set_missing_ages(data_train)
data_train = set_Cabin_type(data_train)
data_train.describe()



data_test.loc[ (data_test.Fare.isnull()), 'Fare' ] = 0
tmp_df = data_test[['Age','Fare', 'Parch', 'SibSp', 'Pclass']]
null_age = tmp_df[data_test.Age.isnull()].as_matrix()
X = null_age[:, 1:]
predictedAges = rfr.predict(X)
data_test.loc[ (data_test.Age.isnull()), 'Age' ] = predictedAges
data_test = set_Cabin_type(data_test)

# data_test, rfr = set_missing_ages(data_test)
# data_test = set_Cabin_type(data_test)

In [42]:
dummies_Cabin = pd.get_dummies(data_train['Cabin'], prefix= 'Cabin')
dummies_Embarked = pd.get_dummies(data_train['Embarked'], prefix= 'Embarked')
dummies_Sex = pd.get_dummies(data_train['Sex'], prefix= 'Sex')
dummies_Pclass = pd.get_dummies(data_train['Pclass'], prefix= 'Pclass')

data_train_adjust = pd.concat([data_train, dummies_Cabin, dummies_Embarked, dummies_Sex, dummies_Pclass], axis=1)
data_train_adjust.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
data_train_adjust

dummies_Cabin = pd.get_dummies(data_test['Cabin'], prefix= 'Cabin')
dummies_Embarked = pd.get_dummies(data_test['Embarked'], prefix= 'Embarked')
dummies_Sex = pd.get_dummies(data_test['Sex'], prefix= 'Sex')
dummies_Pclass = pd.get_dummies(data_test['Pclass'], prefix= 'Pclass')

data_test_adjust = pd.concat([data_test, dummies_Cabin, dummies_Embarked, dummies_Sex, dummies_Pclass], axis=1)
data_test_adjust.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)

In [43]:
import sklearn.preprocessing as preprocessing
scaler = preprocessing.StandardScaler()
age_scale_param = scaler.fit(data_train_adjust['Age'])
data_train_adjust['Age_scaled'] = scaler.fit_transform(data_train_adjust['Age'], age_scale_param)
fare_scale_param = scaler.fit(df['Fare'])
data_train_adjust['Fare_scaled'] = scaler.fit_transform(data_train_adjust['Fare'], fare_scale_param)
data_train_adjust


scaler = preprocessing.StandardScaler()
age_scale_param = scaler.fit(data_test_adjust['Age'])
data_test_adjust['Age_scaled'] = scaler.fit_transform(data_test_adjust['Age'], age_scale_param)
fare_scale_param = scaler.fit(df['Fare'])
data_test_adjust['Fare_scaled'] = scaler.fit_transform(data_test_adjust['Fare'], fare_scale_param)

/home/nbuser/anaconda3_431/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/nbuser/anaconda3_431/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/nbuser/anaconda3_431/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(

In [54]:
data_train_adjust

,PassengerId,Survived,Age,SibSp,Parch,Fare,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Age_scaled,Fare_scaled
0,1,0,22.000000,1,0,7.2500,1,0,0,0,1,0,1,0,0,1,-0.561363,-0.502445
1,2,1,38.000000,1,0,71.2833,0,1,1,0,0,1,0,1,0,0,0.613182,0.786845
2,3,1,26.000000,0,0,7.9250,1,0,0,0,1,1,0,0,0,1,-0.267727,-0.488854
3,4,1,35.000000,1,0,53.1000,0,1,0,0,1,1,0,1,0,0,0.392955,0.420730
4,5,0,35.000000,0,0,8.0500,1,0,0,0,1,0,1,0,0,1,0.392955,-0.486337
5,6,0,23.828953,0,0,8.4583,1,0,0,1,0,0,1,0,0,1,-0.427102,-0.478116
6,7,0,54.000000,0,0,51.8625,0,1,0,0,1,0,1,1,0,0,1.787727,0.395814
7,8,0,2.000000,3,1,21.0750,1,0,0,0,1,0,1,0,0,1,-2.029545,-0.224083
8,9,1,27.000000,0,2,11.1333,1,0,0,0,1,1,0,0,0,1,-0.194318,-0.424256
9,10,1,14.000000,1,0,30.0708,1,0,1,0,0,1,0,0,1,0,-1.148636,-0.042956


In [44]:
def one_hot(arr):
    minVal = min(arr)
    maxVal = max(arr)
    rangeVal = maxVal - minVal + 1
    result = []
    for val in arr:
        tempArr = [0] * int(rangeVal)
        tempArr[int(val - minVal)] = 1
        result.append(tempArr)
    return result
    
    
train_df = data_train_adjust.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*')
train_np = train_df.as_matrix()

trainData = train_np[:, 1:]
trainLabel = one_hot(train_np[:, 0])

test_df = data_test_adjust.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*')
test_np = test_df.as_matrix()

testData = test_np[:, 0:]

print(trainData[0:3])
print(trainLabel[0:3])
print(len(trainData))
print(len(trainData[0]))
print(len(trainLabel))
print(len(trainLabel[0]))
print(testData[0:3])
print(len(testData))
print(len(testData[0]))

[[ 1.          0.          1.          0.          0.          0.          1.
   0.          1.          0.          0.          1.         -0.56136323
  -0.50244517]
 [ 1.          0.          0.          1.          1.          0.          0.
   1.          0.          1.          0.          0.          0.61318183
   0.78684529]
 [ 0.          0.          1.          0.          0.          0.          1.
   1.          0.          0.          0.          1.         -0.26772697
  -0.48885426]]
[[1, 0], [0, 1], [0, 1]]
891
14
891
2
[[ 0.          0.          1.          0.          0.          1.          0.
   0.          1.          0.          0.          1.          0.30753461
  -0.49663711]
 [ 1.          0.          1.          0.          0.          0.          1.
   1.          0.          0.          0.          1.          1.25623007
  -0.5114971 ]
 [ 0.          0.          1.          0.          0.          1.          0.
   0.          1.          0.          1.       

In [60]:
train_data = trainData[:600, :]
train_label = trainLabel[:600]

validate_data = trainData[600:, :]
validate_label = trainLabel[600:]

print(len(train_data))
print(len(train_data[0]))
print(len(train_label))
print(len(train_label[0]))


print(len(validate_data))
print(len(validate_data[0]))
print(len(validate_label))
print(len(validate_label[0]))


600
14
600
2
291
14
291
2


In [62]:
import numpy as np
import tensorflow as tf
import operator

train_data = trainData
train_label = trainLabel

learning_rate = 0.001
atttribueNum = 14
x = tf.placeholder(tf.float32, [None, atttribueNum])
W = tf.Variable(tf.zeros([atttribueNum, 2]))
b = tf.Variable(tf.zeros([2]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 2])
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
print('GradientDescentOptimizer')
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(6):
    start = i*100
    end = (i+1)*100
    subData = train_data[start:end, :]
    subLabel = train_label[start:end]
    sess.run(train_step, feed_dict = {x: subData, y_: subLabel})
    print('train: ', sess.run(cross_entropy, feed_dict={x: train_data, y_: train_label}))
    print('validate: ',sess.run(cross_entropy, feed_dict={x: validate_data, y_: validate_label}))


GradientDescentOptimizer
train:  0.693084
validate:  0.693079
train:  0.692997
validate:  0.692981
train:  0.69293
validate:  0.692903
train:  0.69287
validate:  0.692834
train:  0.692801
validate:  0.692755
train:  0.692745
validate:  0.692691


In [56]:
def reverse_one_hot(matrix):
    result = list()
    for arr in matrix:
        max_index, max_value = max(enumerate(arr), key=operator.itemgetter(1))
        result.append(max_index)
    return result

trainResult = reverse_one_hot(sess.run(y, feed_dict={x: train_data}))
testResult = reverse_one_hot(sess.run(y, feed_dict={x: testData}))

trainMatrix = data_train_adjust.as_matrix()
trainId = trainMatrix[:, 0]

testMatrix = data_test_adjust.as_matrix()
testId = testMatrix[:, 0]

trainDataSet = list(zip(trainId, trainResult))
df = pd.DataFrame(data = trainDataSet, columns=['PassengerId', 'Survived']) 
df.to_csv('train_result.csv', index=False, header=True) 

testDataSet = list(zip(testId, testResult))
df = pd.DataFrame(data = testDataSet, columns=['PassengerId', 'Survived']) 
df.to_csv('test_result.csv', index=False, header=True) 